![msaberpro.png](saberpro.png)

# Modelado - Proyecto Visualización y Storytelling

In [1]:
# Paquetes necesarios
import pandas as pd
import numpy as np
from zipfile import ZipFile #para descomprimir los archivos
import os #para usar la funcion listdir() y crear una lista del directorio
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import table
from datetime import datetime, date

In [2]:
## Datos icfes - clasificacion de planteles
### https://icfesgovco-my.sharepoint.com/personal/dataicfes_icfes_gov_co/_layouts/15/onedrive.aspx?ct=1589295914045&or=OWA%2DNT&cid=27943174%2D8b52%2Deb60%2Dc2bf%2Dfd8870b02519&ga=1
### contraseña: Icfes2020*

RUTA = r"Datos"
ARCHIVOS = os.listdir(RUTA) ## lista donde se guardaran los nombres de los archivos con las bases de datos
# len(ARCHIVOS)#, ARCHIVOS #numero de archivos en formato zip

## Funcion para Extraer los archivos de cada zip a una carpeta temporal
def extraer_a_temporal(ARCHIVOS, RUTA):
    # Para cada zip que se encuentra en la lista de ARCHIVOS
    for archivo in ARCHIVOS:
        # Lee el contenido de cada zip y crea una lista de los archivos que contiene
        with ZipFile(RUTA+"/"+archivo, 'r') as archivo_2:
            name_archivo = archivo_2.namelist()
            # Lee cada archivo dentro del zip y si termina en .txt entonces lo extrae en la carpeta BASES
            for nombre in name_archivo:
                if nombre.endswith('.txt') or nombre.endswith('.TXT') or nombre.endswith('.csv'):
                    archivo_2.extract(nombre, 'BASES')
    return

def importar(RUTA_2,SEPARADOR,name_archivos, enc = 'Windows-1252'):
#     name_archivos = os.listdir(RUTA_2)
    lista_archivos = []
    for name in name_archivos:
        if name.endswith('.txt') or name.endswith('.TXT') or name.endswith('.csv'):
            data = pd.read_csv(RUTA_2+'/'+name, sep = SEPARADOR, engine = 'python',encoding = enc)
            data["Fuente"] = name
            lista_archivos.append(data)
        
    icfesDF = pd.concat(lista_archivos, ignore_index = True)
    return icfesDF

In [3]:
## Ejecutar la función para extraer los archivos a una carpeta temporal
extraer_a_temporal(ARCHIVOS, RUTA)

In [4]:
RUTA_2 = os.path.abspath(os.getcwd())+'\\'+'BASES'
# os.listdir(RUTA_2)
name_archivos_2 = ['SB11-CLASIFI-PLANTELES-20182.txt', 'SB11-CLASIFI-PLANTELES-20191.txt',
                   'SB11-CLASIFI-PLANTELES-20194.txt', 'SB11-CLASIFI-PLANTELES-20201.txt',
                   'SB11-CLASIFI-PLANTELES-20204.txt', 'SB11-CLASIFI-PLANTELES-20211.txt'] # sep = '¬' '|',encoding = 'utf-8'
name_archivos_3 = ['SB11-CLASIFI-PLANTELES-20171.csv','SB11-CLASIFI-PLANTELES-20172.csv'] # sep = '¬' '|',encoding = 'utf-8

## Importar 20181
icfes_1 = pd.read_csv(RUTA_2+'/'+'SB11-CLASIFI-PLANTELES-20181.txt', sep = '\t',engine = 'python', encoding = 'Windows-1252')
icfes_1 = icfes_1.loc[:, ~icfes_1.columns.str.startswith('|')]
icfes_1["Fuente"] = 'SB11-CLASIFI-PLANTELES-20181.txt'

# Importar archivos con caracteristicas similares de sep y encoding
SEPARADOR_2 = '[|¬]'
icfes_2 = importar(RUTA_2,SEPARADOR_2,name_archivos_2, enc = 'utf-8')

# Importar archivos con caracteristicas similares de sep y encoding
SEPARADOR_3 = ','
icfes_3 = importar(RUTA_2,SEPARADOR_3,name_archivos_3, enc = 'utf-8')

## Crear un solo dataframe
icfes = pd.concat([icfes_1, icfes_2, icfes_3], ignore_index = True)
print(f'La base de datos tiene información de {len(icfes["Fuente"].unique())} archivos y {len(icfes.columns)} columnas.')

La base de datos tiene información de 9 archivos y 21 columnas.


Se cargaron 9 archivos de la base de clasificación de planteles del ICFES de los años 2017-2021; para cada año se cargaron dos periodos que corresponden a las dos fechas en las que se desarrolla el examen del icfes durante cada año, a excepción del año 2021 para el que se tiene solo 1 periodo.

In [5]:
## Crear un data frame con la información de los periodos a partir de los nombres de los archivos
# icfes["Fuente"].unique() #Nombres de los archivos
Orden_fuente = [['SB11-CLASIFI-PLANTELES-20171.csv', 2017],
                ['SB11-CLASIFI-PLANTELES-20172.csv', 2017],
                ['SB11-CLASIFI-PLANTELES-20181.txt', 2018],
                ['SB11-CLASIFI-PLANTELES-20182.txt', 2018],
                ['SB11-CLASIFI-PLANTELES-20191.txt', 2019],
                ['SB11-CLASIFI-PLANTELES-20194.txt', 2019],
                ['SB11-CLASIFI-PLANTELES-20201.txt', 2020],
                ['SB11-CLASIFI-PLANTELES-20204.txt', 2020],
                ['SB11-CLASIFI-PLANTELES-20211.txt', 2021]]

fuenteDF = pd.DataFrame(Orden_fuente, columns = ['Fuente', 'Anio'])
# fuenteDF

## Se unen la BD con el nuevo DF para una mejor identificacion de los registros segun el periodo
icfes = icfes.merge(fuenteDF, how = "left", on = "Fuente")

print(f"La base de datos tiene {icfes.shape[0]} registros y {icfes.shape[1]} columnas/variables, \
que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo', 'Anio', 'Sort_global'.")

print(f"Además, tenemos información de 5 años y 9 periodos. \n \
\n {icfes.groupby(by = ['Anio']).size()}")

La base de datos tiene 49191 registros y 22 columnas/variables, que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo', 'Anio', 'Sort_global'.
Además, tenemos información de 5 años y 9 periodos. 
 
 Anio
2017     9362
2018     9431
2019     9181
2020     9156
2021    12061
dtype: int64


In [6]:
icfes.columns

Index(['PERIODO', 'COLE_COD_DANE', 'COLE_INST_NOMBRE', 'COLE_CODMPIO_COLEGIO',
       'COLE_MPIO_MUNICIPIO', 'COLE_COD_DEPTO', 'COLE_DEPTO_COLEGIO',
       'COLE_NATURALEZA', 'COLE_GRADO', 'COLE_CALENDARIO_COLEGIO',
       'COLE_GENEROPOBLACION', 'MATRICULADOS_ULTIMOS_3', 'EVALUADOS_ULTIMOS_3',
       'INDICE_MATEMATICAS', 'INDICE_C_NATURALES',
       'INDICE_SOCIALES_CIUDADANAS', 'INDICE_LECTURA_CRITICA', 'INDICE_INGLES',
       'INDICE_TOTAL', 'COLE_CATEGORIA', 'Fuente', 'Anio'],
      dtype='object')

In [7]:
## Obtener la base de datos con las columnas de interes
# list(icfes.columns.values)
lista_columnas = ['COLE_COD_DANE','COLE_INST_NOMBRE', 'COLE_CODMPIO_COLEGIO', 'COLE_MPIO_MUNICIPIO',
                  'COLE_DEPTO_COLEGIO', 'COLE_NATURALEZA', 'COLE_CALENDARIO_COLEGIO', 'COLE_CATEGORIA', 
                  'INDICE_MATEMATICAS', 'INDICE_C_NATURALES', 'INDICE_SOCIALES_CIUDADANAS', 
                  'INDICE_LECTURA_CRITICA', 'INDICE_INGLES','INDICE_TOTAL','Anio']
icfes_final = icfes.loc[:, lista_columnas]

print(f"Después de quedarnos solo con la información de interes, la nueva base de \
datos tiene {icfes_final.shape[0]} registros (se eliminaron {icfes.shape[0] - icfes_final.shape[0]} registros) y {icfes_final.shape[1]} columnas/variables, \
que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo'.")
display(icfes_final)

## Verificando los valores missing
## Creando el porcentaje de valores missing de toda la base
null_columnas = icfes_final.columns[icfes_final.isnull().any()]
name_columnas_null = icfes_final[null_columnas].isnull().sum() *100 / icfes_final.shape[0]
name_columnas_null = name_columnas_null.sort_values(ascending = False)

print(f"En general, hay variables ({len(name_columnas_null)} de las {icfes_final.shape[1]}) con missing values.")
icfes_final = icfes_final.dropna(how='any')
## Tipos de variables en la base de datos (hasta este momento la BD mantiene los mismos indices desde su importación)
icfes_final[['COLE_COD_DANE','Otro']] = icfes_final["COLE_COD_DANE"].astype('str').str.split(".",expand=True)
icfes_final.drop(['Otro'], axis=1, inplace=True)
icfes_final.info()

Después de quedarnos solo con la información de interes, la nueva base de datos tiene 49191 registros (se eliminaron 0 registros) y 15 columnas/variables, que incluyen las nuevas variables identificadoras que fueron agregadas: 'Periodo'.


COLE_COD_DANE                                   COLE_INST_NOMBRE  \
0       3.080011e+11                                    AMERICAN SCHOOL   
1       3.085730e+11  INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...   
2       3.080011e+11                                   COLEGIO ALTAMIRA   
3       3.080011e+11                    COLEGIO BRITANICO INTERNACIONAL   
4       3.085731e+11                                   COLEGIO SAN JOSÉ   
...              ...                                                ...   
49186   2.231890e+11           INSTITUCION EDUCATIVA LOS MIMBRES CENTRO   
49187   3.110010e+11  COL CRISTIANO PSICOPEDAG SINAI                ...   
49188   3.202500e+11                   COLEGIO NUESTRA SEÑORA DE FATIMA   
49189   3.200010e+11                             LICEO CRISTIANO LA PAZ   
49190   2.195170e+11                               INST EDUC LA MURALLA   

       COLE_CODMPIO_COLEGIO COLE_MPIO_MUNICIPIO COLE_DEPTO_COLEGIO  \
0                      8573     PUERTO COLOMBIA          ATLANTICO   
1                      8573     PUERTO COLOMBIA          ATLANTICO   
2                      8573     PUERTO COLOMBIA          ATLANTICO   
3                      8573     PUERTO COLOMBIA          ATLANTICO   
4                      8573     PUERTO COLOMBIA          ATLANTICO   
...                     ...                 ...                ...   
49186                 23189      CIENAGA DE ORO            CORDOBA   
49187                 11001         BOGOTÁ D.C.             BOGOTA   
49188                 20250             EL PASO              CESAR   
49189                 20001          VALLEDUPAR              CESAR   
49190                 19517   PAEZ (BELALCAZAR)              CAUCA   

      COLE_NATURALEZA COLE_CALENDARIO_COLEGIO COLE_CATEGORIA  \
0          NO OFICIAL                       B             A+   
1          NO OFICIAL                       B             A+   
2          NO OFICIAL                       B             A+   
3          NO OFICIAL                       B             A+   
4          NO OFICIAL                       B             A+   
...               ...                     ...            ...   
49186         OFICIAL                       A              D   
49187      NO OFICIAL                       A              A   
49188      NO OFICIAL                       A              D   
49189      NO OFICIAL                       A              D   
49190         OFICIAL                       A              D   

      INDICE_MATEMATICAS INDICE_C_NATURALES INDICE_SOCIALES_CIUDADANAS  \
0                 0.8184             0.8297                     0.8215   
1                 0.8173             0.8321                     0.8212   
2                  0.887             0.8848                     0.8731   
3                 0.8626             0.8581                     0.8374   
4                 0.8541             0.8493                     0.8423   
...                  ...                ...                        ...   
49186             0.5149              0.585                     0.5787   
49187             0.7162             0.7366                     0.7367   
49188             0.5596              0.604                     0.5564   
49189             0.5401             0.6237                     0.6258   
49190              0.427             0.4979                     0.4765   

      INDICE_LECTURA_CRITICA INDICE_INGLES INDICE_TOTAL  Anio  
0                     0.8145        0.9203       0.8287  2018  
1                     0.8125        0.9068       0.8274  2018  
2                     0.8633        0.9562       0.8832  2018  
3                     0.8402        0.9496       0.8573  2018  
4                     0.8378        0.9217       0.8517  2018  
...                      ...           ...          ...   ...  
49186                 0.5944        0.5451       0.5665  2017  
49187                 0.7453        0.6957       0.7308  2017  
49188                 0.6041   

En general, hay variables (1 de las 15) con missing values.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49172 entries, 0 to 49190
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   COLE_COD_DANE               49172 non-null  object
 1   COLE_INST_NOMBRE            49172 non-null  object
 2   COLE_CODMPIO_COLEGIO        49172 non-null  int64 
 3   COLE_MPIO_MUNICIPIO         49172 non-null  object
 4   COLE_DEPTO_COLEGIO          49172 non-null  object
 5   COLE_NATURALEZA             49172 non-null  object
 6   COLE_CALENDARIO_COLEGIO     49172 non-null  object
 7   COLE_CATEGORIA              49172 non-null  object
 8   INDICE_MATEMATICAS          49172 non-null  object
 9   INDICE_C_NATURALES          49172 non-null  object
 10  INDICE_SOCIALES_CIUDADANAS  49172 non-null  object
 11  INDICE_LECTURA_CRITICA      49172 non-null  object
 12  INDICE_INGLES               49172 non-null

In [8]:
## Estadísticas descriptivas de los estudiantes
print(f"A continuación se presenta para las variables categoricas del estudiante, el número de observaciones, \
los dominios, la moda y el número de veces que se repite la moda en la base de datos.")
icfes_final.describe(include='object').transpose()

A continuación se presenta para las variables categoricas del estudiante, el número de observaciones, los dominios, la moda y el número de veces que se repite la moda en la base de datos.


count unique                                  top  \
COLE_COD_DANE               49172  12970                         311001000000   
COLE_INST_NOMBRE            49172  12419  INSTITUCION EDUCATIVA SIMON BOLIVAR   
COLE_MPIO_MUNICIPIO         49172   1409                          BOGOTÁ D.C.   
COLE_DEPTO_COLEGIO          49172     34                            ANTIOQUIA   
COLE_NATURALEZA             49172      2                              OFICIAL   
COLE_CALENDARIO_COLEGIO     49172      3                                    A   
COLE_CATEGORIA              49172      5                                    D   
INDICE_MATEMATICAS          49172   8160                               0,7078   
INDICE_C_NATURALES          49172   7399                               0,6683   
INDICE_SOCIALES_CIUDADANAS  49172   7857                               0,6535   
INDICE_LECTURA_CRITICA      49172   6869                               0,7224   
INDICE_INGLES               49172   8278                               0,6569   
INDICE_TOTAL                49172   7572                               0,6457   

                             freq  
COLE_COD_DANE                 537  
COLE_INST_NOMBRE               83  
COLE_MPIO_MUNICIPIO          5433  
COLE_DEPTO_COLEGIO           5562  
COLE_NATURALEZA             34158  
COLE_CALENDARIO_COLEGIO     47430  
COLE_CATEGORIA              14351  
INDICE_MATEMATICAS             33  
INDICE_C_NATURALES             33  
INDICE_SOCIALES_CIUDADANAS     40  
INDICE_LECTURA_CRITICA         40  
INDICE_INGLES                  31  
INDICE_TOTAL                   35

In [9]:
icfes_final.groupby(['COLE_CALENDARIO_COLEGIO'])['COLE_CALENDARIO_COLEGIO'].count()

COLE_CALENDARIO_COLEGIO
A    47430
B     1676
O       66
Name: COLE_CALENDARIO_COLEGIO, dtype: int64

Solo vamos a seleccionar los colegios con calendario A y B, así que prescindiremos de los colegios de calendario O (flexible u otro).

In [10]:
icfes_final = icfes_final[icfes_final['COLE_CALENDARIO_COLEGIO'] != 'O']
icfes_final.shape

(49106, 15)

Ahora se modifican los nombres de las variables para que sean faciles de usar en la visualización

In [11]:
# Diccionario con los nombres de las variables (old) y los nuevos nombres
# list(icfes_final.columns)
names_old_new = {'COLE_INST_NOMBRE' : 'Nombre_Colegio',
                 'COLE_CODMPIO_COLEGIO' : 'Cod_Municipio',
                 'COLE_MPIO_MUNICIPIO' : 'Municipio',
                 'COLE_DEPTO_COLEGIO' : 'Departamento',
                 'COLE_NATURALEZA' : 'Tipo',
                 'COLE_CALENDARIO_COLEGIO' : 'Calendario',
                 'COLE_CATEGORIA' : 'Clasificacion',
                 'Anio' : 'Anio_prueba'
                 }
icfes_final = icfes_final.rename(columns = names_old_new, inplace=False)
icfes_final.head()

COLE_COD_DANE                                     Nombre_Colegio  \
0  308001074789                                    AMERICAN SCHOOL   
1  308573000450  INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...   
2  308001073952                                   COLEGIO ALTAMIRA   
3  308001101153                    COLEGIO BRITANICO INTERNACIONAL   
4  308573074909                                   COLEGIO SAN JOSÉ   

   Cod_Municipio        Municipio Departamento        Tipo Calendario  \
0           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL          B   
1           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL          B   
2           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL          B   
3           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL          B   
4           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL          B   

  Clasificacion INDICE_MATEMATICAS INDICE_C_NATURALES  \
0            A+             0.8184             0.8297   
1            A+             0.8173             0.8321   
2            A+              0.887             0.8848   
3            A+             0.8626             0.8581   
4            A+             0.8541             0.8493   

  INDICE_SOCIALES_CIUDADANAS INDICE_LECTURA_CRITICA INDICE_INGLES  \
0                     0.8215                 0.8145        0.9203   
1                     0.8212                 0.8125        0.9068   
2                     0.8731                 0.8633        0.9562   
3                     0.8374                 0.8402        0.9496   
4                     0.8423                 0.8378        0.9217   

  INDICE_TOTAL  Anio_prueba  
0       0.8287         2018  
1       0.8274         2018  
2       0.8832         2018  
3       0.8573         2018  
4       0.8517         2018

Verificar los nombres de los municipios, de los departamentos y de los colegios que requieren algún tipo de estandarización.

In [12]:
print(f'Valores unicos de Cod_Municipio: {len(icfes_final["Cod_Municipio"].unique())}')
print(f'Valores unicos de Municipio: {len(icfes_final["Municipio"].unique())}')
print(f'Valores unicos de Departamento: {len(icfes_final["Departamento"].unique())}')
print(f'Valores unicos de COLE_COD_DANE: {len(icfes_final["COLE_COD_DANE"].unique())}')
print(f'Valores unicos de Nombre_Colegio: {len(icfes_final["Nombre_Colegio"].unique())}')

Valores unicos de Cod_Municipio: 1116
Valores unicos de Municipio: 1409
Valores unicos de Departamento: 34
Valores unicos de COLE_COD_DANE: 12930
Valores unicos de Nombre_Colegio: 12388


In [13]:
## Se tienen dos codigos de municipio que no corresponden
# icfes_final[(icfes_final['Cod_Municipio'] == 27086) | (icfes_final['Cod_Municipio'] == 94663)]
icfes_final['Cod_Municipio'] = icfes_final['Cod_Municipio'].replace(94663, 94343)
icfes_final['Cod_Municipio'] = icfes_final['Cod_Municipio'].replace(27086, 27615)
# 94663 = 94343
# 27086 = 27615

## Seleccionar el codigo y el nombre del colegio para unificar y eliminar los duplicados
icfes_code_colegio = icfes_final[['COLE_COD_DANE','Nombre_Colegio']]
icfes_code_colegio.drop_duplicates(subset=['COLE_COD_DANE'], inplace=True, ignore_index = True)

## Eliminar la variable nombre colegio y hacer el merge con los codigos sin duplicados
icfes_final = icfes_final.drop(['Nombre_Colegio'], axis = 1)
df_final = icfes_final.merge(icfes_code_colegio, how = 'left', on = 'COLE_COD_DANE')
df_final.head()

C:\Users\jesus\AppData\Local\Temp\ipykernel_16796\1691066868.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icfes_code_colegio.drop_duplicates(subset=['COLE_COD_DANE'], inplace=True, ignore_index = True)


COLE_COD_DANE  Cod_Municipio        Municipio Departamento        Tipo  \
0  308001074789           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL   
1  308573000450           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL   
2  308001073952           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL   
3  308001101153           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL   
4  308573074909           8573  PUERTO COLOMBIA    ATLANTICO  NO OFICIAL   

  Calendario Clasificacion INDICE_MATEMATICAS INDICE_C_NATURALES  \
0          B            A+             0.8184             0.8297   
1          B            A+             0.8173             0.8321   
2          B            A+              0.887             0.8848   
3          B            A+             0.8626             0.8581   
4          B            A+             0.8541             0.8493   

  INDICE_SOCIALES_CIUDADANAS INDICE_LECTURA_CRITICA INDICE_INGLES  \
0                     0.8215                 0.8145        0.9203   
1                     0.8212                 0.8125        0.9068   
2                     0.8731                 0.8633        0.9562   
3                     0.8374                 0.8402        0.9496   
4                     0.8423                 0.8378        0.9217   

  INDICE_TOTAL  Anio_prueba                                     Nombre_Colegio  
0       0.8287         2018                                    AMERICAN SCHOOL  
1       0.8274         2018  INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...  
2       0.8832         2018                                   COLEGIO ALTAMIRA  
3       0.8573         2018                    COLEGIO BRITANICO INTERNACIONAL  
4       0.8517         2018                                   COLEGIO SAN JOSÉ

In [14]:
print(f'Valores unicos de COLE_COD_DANE: {len(df_final["COLE_COD_DANE"].unique())}')
print(f'Valores unicos de Nombre_Colegio: {len(df_final["Nombre_Colegio"].unique())}')

Valores unicos de COLE_COD_DANE: 12930
Valores unicos de Nombre_Colegio: 9921


Teniendo en cuenta que no se encuentran estandarizados los municipios y departamentos, se procede a estandarizar con base en los codigos divipola del DANE.

In [15]:
## Cargar dataframe con el que se va a solucionar
df_base = pd.read_excel('Intermedia/Listados_DIVIPOLA.xlsx', sheet_name = 'Municipios', skiprows = 10, nrows = 1121)
names_old_new = {'Código .1' : 'Cod_Municipio',
                 'Nombre.1' : 'Municipio',
                 'Nombre' : 'Departamento'
                }
df_base = df_base.rename(columns = names_old_new, inplace=False)

## Seleccionar columnas de interes
lista_columnas = ['Cod_Municipio','Municipio', 'Departamento']
df_base = df_base.loc[:, lista_columnas]

## Eliminar la variable municipio y departamento y hacer el merge con los codigos sin duplicados
df_final = df_final.drop(['Municipio','Departamento'], axis = 1)
df_final = df_final.merge(df_base, how = 'left', on = 'Cod_Municipio')
display(df_final)

print(f'Valores unicos de Cod_Municipio: {len(df_final["Cod_Municipio"].unique())}')
print(f'Valores unicos de Municipio: {len(df_final["Municipio"].unique())}')
print(f'Valores unicos de Departamento: {len(df_final["Departamento"].unique())}')

COLE_COD_DANE  Cod_Municipio        Tipo Calendario Clasificacion  \
0      308001074789           8573  NO OFICIAL          B            A+   
1      308573000450           8573  NO OFICIAL          B            A+   
2      308001073952           8573  NO OFICIAL          B            A+   
3      308001101153           8573  NO OFICIAL          B            A+   
4      308573074909           8573  NO OFICIAL          B            A+   
...             ...            ...         ...        ...           ...   
49101  223189000000          23189     OFICIAL          A             D   
49102  311001000000          11001  NO OFICIAL          A             A   
49103  320250000000          20250  NO OFICIAL          A             D   
49104  320001000000          20001  NO OFICIAL          A             D   
49105  219517000000          19517     OFICIAL          A             D   

      INDICE_MATEMATICAS INDICE_C_NATURALES INDICE_SOCIALES_CIUDADANAS  \
0                 0.8184             0.8297                     0.8215   
1                 0.8173             0.8321                     0.8212   
2                  0.887             0.8848                     0.8731   
3                 0.8626             0.8581                     0.8374   
4                 0.8541             0.8493                     0.8423   
...                  ...                ...                        ...   
49101             0.5149              0.585                     0.5787   
49102             0.7162             0.7366                     0.7367   
49103             0.5596              0.604                     0.5564   
49104             0.5401             0.6237                     0.6258   
49105              0.427             0.4979                     0.4765   

      INDICE_LECTURA_CRITICA INDICE_INGLES INDICE_TOTAL  Anio_prueba  \
0                     0.8145        0.9203       0.8287         2018   
1                     0.8125        0.9068       0.8274         2018   
2                     0.8633        0.9562       0.8832         2018   
3                     0.8402        0.9496       0.8573         2018   
4                     0.8378        0.9217       0.8517         2018   
...                      ...           ...          ...          ...   
49101                 0.5944        0.5451       0.5665         2017   
49102                 0.7453        0.6957       0.7308         2017   
49103                 0.6041        0.6114       0.5833         2017   
49104                 0.6593        0.5824       0.6099         2017   
49105                 0.4983        0.5533        0.481         2017   

                                          Nombre_Colegio        Municipio  \
0                                        AMERICAN SCHOOL  PUERTO COLOMBIA   
1      INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...  PUERTO COLOMBIA   
2                                       COLEGIO ALTAMIRA  PUERTO COLOMBIA   
3                        COLEGIO BRITANICO INTERNACIONAL  PUERTO COLOMBIA   
4                                       COLEGIO SAN JOSÉ  PUERTO COLOMBIA   
...                                                  ...              ...   
49101                         I.E. SAN JOSE DE LA GUNETA   CIÉNAGA DE ORO   
49102                                COL BILING RICHMOND     BOGOTÁ, D.C.   
49103                   COLEGIO NUESTRA SEÑORA DE FATIMA          EL PASO   
49104                                COLEGIO SAN ANTONIO       VALLEDUPAR   
49105              INSTITUCION EDUCATIVA JOSE REYES PETE             PÁEZ   

       Departamento  
0         ATLÁNTICO  
1         ATLÁNTICO  
2         ATLÁNTICO  
3         ATLÁNTICO  
4         ATLÁNTICO  
...             ...  
49101       CÓRDOBA  
49102  BOGOTÁ, D.C.  
49103         CESAR  
49104         CESAR  
49105         CAUCA  

[49106 rows x 15 columns]

Valores unicos de Cod_Municipio: 1114
Valores unicos de Municipio: 1031
Valores unicos de Departamento: 33


In [16]:
df_final = df_final.drop(['COLE_COD_DANE'], axis=1)
df_final.to_csv('Base_final_icfes.csv')
df_final

Cod_Municipio        Tipo Calendario Clasificacion INDICE_MATEMATICAS  \
0               8573  NO OFICIAL          B            A+             0.8184   
1               8573  NO OFICIAL          B            A+             0.8173   
2               8573  NO OFICIAL          B            A+              0.887   
3               8573  NO OFICIAL          B            A+             0.8626   
4               8573  NO OFICIAL          B            A+             0.8541   
...              ...         ...        ...           ...                ...   
49101          23189     OFICIAL          A             D             0.5149   
49102          11001  NO OFICIAL          A             A             0.7162   
49103          20250  NO OFICIAL          A             D             0.5596   
49104          20001  NO OFICIAL          A             D             0.5401   
49105          19517     OFICIAL          A             D              0.427   

      INDICE_C_NATURALES INDICE_SOCIALES_CIUDADANAS INDICE_LECTURA_CRITICA  \
0                 0.8297                     0.8215                 0.8145   
1                 0.8321                     0.8212                 0.8125   
2                 0.8848                     0.8731                 0.8633   
3                 0.8581                     0.8374                 0.8402   
4                 0.8493                     0.8423                 0.8378   
...                  ...                        ...                    ...   
49101              0.585                     0.5787                 0.5944   
49102             0.7366                     0.7367                 0.7453   
49103              0.604                     0.5564                 0.6041   
49104             0.6237                     0.6258                 0.6593   
49105             0.4979                     0.4765                 0.4983   

      INDICE_INGLES INDICE_TOTAL  Anio_prueba  \
0            0.9203       0.8287         2018   
1            0.9068       0.8274         2018   
2            0.9562       0.8832         2018   
3            0.9496       0.8573         2018   
4            0.9217       0.8517         2018   
...             ...          ...          ...   
49101        0.5451       0.5665         2017   
49102        0.6957       0.7308         2017   
49103        0.6114       0.5833         2017   
49104        0.5824       0.6099         2017   
49105        0.5533        0.481         2017   

                                          Nombre_Colegio        Municipio  \
0                                        AMERICAN SCHOOL  PUERTO COLOMBIA   
1      INSTITUCION EDUCATIVA ASPAEN GIMNASIO ALTAMAR ...  PUERTO COLOMBIA   
2                                       COLEGIO ALTAMIRA  PUERTO COLOMBIA   
3                        COLEGIO BRITANICO INTERNACIONAL  PUERTO COLOMBIA   
4                                       COLEGIO SAN JOSÉ  PUERTO COLOMBIA   
...                                                  ...              ...   
49101                         I.E. SAN JOSE DE LA GUNETA   CIÉNAGA DE ORO   
49102                                COL BILING RICHMOND     BOGOTÁ, D.C.   
49103                   COLEGIO NUESTRA SEÑORA DE FATIMA          EL PASO   
49104                                COLEGIO SAN ANTONIO       VALLEDUPAR   
49105              INSTITUCION EDUCATIVA JOSE REYES PETE             PÁEZ   

       Departamento  
0         ATLÁNTICO  
1         ATLÁNTICO  
2         ATLÁNTICO  
3         ATLÁNTICO  
4         ATLÁNTICO  
...             ...  
49101       CÓRDOBA  
49102  BOGOTÁ, D.C.  
49103         CESAR  
49104         CESAR  
49105         CAUCA  

[49106 rows x 14 columns]